In [16]:
import numpy as np
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import Flatten
from keras.layers import Dropout
from keras.layers import BatchNormalization
from keras.layers import Activation
from keras.layers.convolutional import Conv2D
from keras.layers.convolutional import MaxPooling2D
from keras.layers.convolutional import ZeroPadding2D
from keras.optimizers import SGD
from keras.utils import np_utils
from keras.preprocessing.image import ImageDataGenerator
from keras.models import load_model


In [17]:
print(device_lib.list_local_devices())

np.random.seed(3)

[name: "/device:CPU:0"
device_type: "CPU"
memory_limit: 268435456
locality {
}
incarnation: 17577066728645420498
, name: "/device:GPU:0"
device_type: "GPU"
memory_limit: 4854028697
locality {
  bus_id: 1
  links {
  }
}
incarnation: 17548956495295955082
physical_device_desc: "device: 0, name: GeForce GTX 1660 Ti, pci bus id: 0000:1f:00.0, compute capability: 7.5"
]


## 8번 문제

AlexNet 사용해서 정답률을 높였다.
overfitting 막기 위해 dropout을 사용하였다.
또한 기존 CNN보다 훨씬 더 깊게 레이어를 둬서 더 중요한 feature들을 추출해낸다.



In [23]:
train_datagen = ImageDataGenerator(rescale=1./255)

train_generator = train_datagen.flow_from_directory(
        './train_galaxy', #파일 경로
        target_size=(50,50),
        color_mode="grayscale",
        batch_size=32,
        class_mode='categorical')

Found 10233 images belonging to 3 classes.


In [24]:
val_datagen = ImageDataGenerator(rescale=1./255)

val_generator = val_datagen.flow_from_directory(
        './val_galaxy', #파일 경로
        target_size=(50,50),
        color_mode="grayscale",
        batch_size=32,
        class_mode='categorical')

Found 600 images belonging to 3 classes.


In [25]:
test_datagen = ImageDataGenerator(rescale=1./255)

test_generator = test_datagen.flow_from_directory(
        './test_galaxy', #파일 경로
        target_size=(50,50),
        color_mode="grayscale",
        batch_size=32,
        class_mode='categorical')

Found 600 images belonging to 3 classes.


In [28]:
with K.tf.device('/gpu:0'):
    model = Sequential()
    #Alexnet - 계층 1 : 11x11 필터를 96개를 사용, strides = 4, 활화화함수 = relu,
    # 입력 데이터 크기 224x224 , 3x3 크기의 풀리계층 사용

    model.add(Conv2D(96, (11,11), strides=4, input_shape=(50,50,1), activation='relu'))
    model.add(MaxPooling2D(pool_size=(2,2),strides=2))
    model.add(BatchNormalization())

    #Alexnet - 계층 2 : 5X5 필터를 256개 사용 , strides = 1, 활화화함수 = relu, 3x3 크기의 풀리계층 사용
    model.add(ZeroPadding2D(2))
    model.add(Conv2D(256,(5,5), strides=1, activation='relu'))

    model.add(MaxPooling2D(pool_size=(2,2),strides=2))
    model.add(BatchNormalization())

    #Alexnet - 계층 3 : 3x3 필터를 384개 사용, strides =1 , 활성화함수 = relu
    model.add(ZeroPadding2D(1))
    model.add(Conv2D(384,(3,3), strides=1, activation='relu'))


    #Alexnet - 계층 4 : 3x3 필터를 384개 사용, strides =1 , 활성화함수 = relu
    model.add(ZeroPadding2D(1))
    model.add(Conv2D(384,(3,3), strides=1, activation='relu'))


    #Alexnet - 계층 5 : 3x3 필터를 256개 사용, strides =1 , 활성화함수 = relu, 3x3 크기의 풀리계층 사용
    model.add(ZeroPadding2D(1))
    model.add(Conv2D(256,(3,3), strides=1, activation='relu'))
    model.add(MaxPooling2D(pool_size=(2,2), strides=2))

    #계산을 위해서 1차원 배열로 전환
    model.add(Flatten())

    #Alexnet - 계층 6 : 4096개의 출력뉴런, 활성화함수 = relu
    model.add(Dense(4096, activation='relu'))
    model.add(Dropout(0.5))

    #Alexnet - 계층 7 : 4096게의 출력뉴런, 활성화함수 = relu
    model.add(Dense(4096, activation='relu'))
    model.add(Dropout(0.5))

    #Alexnet - 계층 8 : 1개의 출력뉴런, 활성화함수 = sigmoid
    model.add(Dense(3, activation='sigmoid'))

    #학습과정 설정 - 손실함수는 크로스엔트로피, 가중치 검색은 아담
    sgd = SGD(lr=0.01,decay=5e-4, momentum=0.9)
    model.compile(loss='binary_crossentropy', optimizer='sgd',metrics=['accuracy'])

    model.summary()

    hist=model.fit_generator(
        train_generator,
        steps_per_epoch=319, 
        epochs=50,
        validation_data=val_generator,
        validation_steps=18)  

Model: "sequential_8"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_34 (Conv2D)           (None, 10, 10, 96)        11712     
_________________________________________________________________
max_pooling2d_19 (MaxPooling (None, 5, 5, 96)          0         
_________________________________________________________________
batch_normalization_10 (Batc (None, 5, 5, 96)          384       
_________________________________________________________________
zero_padding2d_17 (ZeroPaddi (None, 9, 9, 96)          0         
_________________________________________________________________
conv2d_35 (Conv2D)           (None, 5, 5, 256)         614656    
_________________________________________________________________
max_pooling2d_20 (MaxPooling (None, 2, 2, 256)         0         
_________________________________________________________________
batch_normalization_11 (Batc (None, 2, 2, 256)        

In [29]:
print("--Evaluate--")
scores = model.evaluate_generator(test_generator, steps=200)
print("%s: %.2f%%" % (model.metrics_names[1], scores[1]*100))

--Evaluate--
accuracy: 87.95%
